In [1]:
import itertools
import logging
from pathlib import Path
import numba as nb

import awkward as ak
import click
import h5py as h5
import numpy as np
import vector

from coffea.hist.plot import clopper_pearson_interval
import matplotlib.pyplot as plt

# from src.data.cms.convert_to_h5 import MIN_JETS, N_JETS, N_FJETS

vector.register_awkward()

logging.basicConfig(level=logging.INFO)

/Users/billyli/miniforge_x86_new/lib/python3.9/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
# read test target file
test_file = "//Users/billyli/UCSD/hhh/reports/bv2/hhh_test.h5"
test_h5 = h5.File(test_file)

# read baseline prediction
baseline_file = "//Users/billyli/UCSD/hhh/reports/bv2/pred_baseline.h5"
b_h5 = h5.File(baseline_file)

# read spanet prediction
spanet_file = "//Users/billyli/UCSD/hhh/reports/bv2/dp_on/pred_v53.h5"
s_h5 = h5.File(spanet_file)

In [3]:
h1ap = np.array(s_h5['TARGETS']['h1']['assignment_probability'])
h2ap = np.array(s_h5['TARGETS']['h2']['assignment_probability'])
h3ap = np.array(s_h5['TARGETS']['h3']['assignment_probability'])

h1dp = np.array(s_h5['TARGETS']['h1']['detection_probability'])
h2dp = np.array(s_h5['TARGETS']['h2']['detection_probability'])
h3dp = np.array(s_h5['TARGETS']['h3']['detection_probability'])

In [4]:
dpMin=0.5
apMin=1/(13*13)
h1mask = (h1dp>dpMin)&(h1ap>apMin)
h2mask = (h2dp>dpMin)&(h2ap>apMin)
h3mask = (h3dp>dpMin)&(h3ap>apMin)

In [5]:
h1b1 = np.array(s_h5['TARGETS']['h1']['b1'])
h2b1 = np.array(s_h5['TARGETS']['h2']['b1'])
h3b1 = np.array(s_h5['TARGETS']['h3']['b1'])

h1b2 = np.array(s_h5['TARGETS']['h1']['b2'])
h2b2 = np.array(s_h5['TARGETS']['h2']['b2'])
h3b2 = np.array(s_h5['TARGETS']['h3']['b2'])

In [6]:
h1b1b2 = np.stack([h1b1, h1b2], axis=1)
h2b1b2 = np.stack([h2b1, h2b2], axis=1)
h3b1b2 = np.stack([h3b1, h3b2], axis=1)

In [7]:
# jc = jet collision
# jpc = jet pair collision
N_event = 0
N_jc_event = 0
N_jpc_event = 0

for h1b1b2_e, h2b1b2_e in zip(h1b1b2[h1mask&h2mask], h2b1b2[h1mask&h2mask]):
    N_event += 1
    
    h1b1b2set = set(h1b1b2_e)
    h2b1b2set = set(h2b1b2_e)
    commonJetIndex = h1b1b2set.intersection(h2b1b2set)
    print(h1b1b2set, h2b1b2set, commonJetIndex)
    N_commonIndex = len(commonJetIndex)
    
    if N_commonIndex == 2:
        N_jpc_event +=1
    elif N_commonIndex == 1:
        N_jc_event += 1

print("jet pair collision event", N_jpc_event, "out of", N_event)
print("jet collision event", N_jc_event, "out of", N_event)

{0, 2} {3, 4} set()
{0, 6} {3, 5} set()
{1, 3} {5, 7} set()
{0, 6} {2, 3} set()
{1, 2} {3, 4} set()
{1, 6} {2, 4} set()
{1, 3} {2, 5} set()
{2, 4} {3, 5} set()
{1, 3} {2, 5} set()
{1, 4} {3, 6} set()
{0, 5} {2, 4} set()
{0, 3} {1, 6} set()
{4, 5} {0, 3} set()
{0, 6} {2, 3} set()
{1, 3} {0, 5} set()
{2, 5} {3, 4} set()
{1, 3} {4, 6} set()
{0, 1} {2, 5} set()
{2, 7} {3, 6} set()
{2, 4} {6, 7} set()
{2, 5} {1, 3} set()
{0, 6} {3, 4} set()
{0, 6} {1, 2} set()
{2, 5} {1, 4} set()
{4, 6} {3, 5} set()
{0, 3} {2, 6} set()
{0, 2} {3, 5} set()
{0, 2} {4, 5} set()
{1, 3} {8, 4} set()
{0, 1} {2, 3} set()
{1, 2} {4, 6} set()
{3, 5} {2, 4} set()
{0, 3} {1, 2} set()
{0, 3} {4, 6} set()
{0, 2} {3, 5} set()
{0, 3} {1, 4} set()
{0, 4} {1, 2} set()
{0, 2} {5, 6} set()
{4, 5} {1, 3} set()
{0, 1} {3, 5} set()
{4, 5} {1, 3} set()
{1, 4} {0, 3} set()
{0, 5} {1, 2} set()
{1, 3} {5, 6} set()
{0, 3} {2, 4} set()
{3, 5} {1, 7} set()
{5, 6} {0, 3} set()
{0, 2} {1, 5} set()
{0, 1} {2, 6} set()
{1, 4} {2, 3} set()


In [8]:
# jc = jet collision
# jpc = jet pair collision
N_event = 0
N_jc_event = 0
N_jpc_event = 0

for h2b1b2_e, h3b1b2_e in zip(h2b1b2[h2mask&h3mask], h3b1b2[h2mask&h3mask]):
    N_event += 1
    
    h2b1b2set = set(h2b1b2_e)
    h3b1b2set = set(h3b1b2_e)
    commonJetIndex = h2b1b2set.intersection(h3b1b2set)
    N_commonIndex = len(commonJetIndex)
    
    if N_commonIndex == 2:
        N_jpc_event +=1
    elif N_commonIndex == 1:
        N_jc_event += 1

print("jet pair collision event", N_jpc_event, "out of", N_event)
print("jet collision event", N_jc_event, "out of", N_event)

jet pair collision event 0 out of 14638
jet collision event 0 out of 14638


In [9]:
# jc = jet collision
# jpc = jet pair collision
N_event = 0
N_jc_event = 0
N_jpc_event = 0

for h1b1b2_e, h3b1b2_e in zip(h1b1b2[h1mask&h3mask], h3b1b2[h1mask&h3mask]):
    N_event += 1
    
    h1b1b2set = set(h1b1b2_e)
    h3b1b2set = set(h3b1b2_e)
    commonJetIndex = h1b1b2set.intersection(h3b1b2set)
    N_commonIndex = len(commonJetIndex)
    
    if N_commonIndex == 2:
        N_jpc_event +=1
    elif N_commonIndex == 1:
        N_jc_event += 1

print("jet pair collision event", N_jpc_event, "out of", N_event)
print("jet collision event", N_jc_event, "out of", N_event)

jet pair collision event 0 out of 12237
jet collision event 0 out of 12237


In [10]:
a = {1,2,3}
b = {3,4,5}
c = {6}
print(a.intersection(b))
print(a.intersection(c))
print(len(a.intersection(b)))

{3}
set()
1


Where are the jet collision events...?

In [11]:
# jc = jet collision
# jpc = jet pair collision
N_event = 0
N_jc_event = 0
N_jpc_event = 0

for h1b1b2_e, h2b1b2_e in zip(h1b1b2, h2b1b2):
    N_event += 1
    
    h1b1b2set = set(h1b1b2_e)
    h2b1b2set = set(h2b1b2_e)
    commonJetIndex = h1b1b2set.intersection(h2b1b2set)
    N_commonIndex = len(commonJetIndex)
    print(h1b1b2set, h2b1b2set)
    
    if N_commonIndex == 2:
        N_jpc_event +=1
    elif N_commonIndex == 1:
        N_jc_event += 1

print("jet pair collision event", N_jpc_event, "out of", N_event)
print("jet collision event", N_jc_event, "out of", N_event)

{5} {6}
{0, 2} {3, 4}
{1, 5} {2, 4}
{0, 1} {6}
{0, 6} {3, 5}
{5, 6} {0, 2}
{1, 3} {5, 7}
{2, 3} {4, 5}
{0, 6} {2, 3}
{1, 2} {3, 4}
{1, 6} {2, 4}
{1, 3} {2, 5}
{7} {5, 6}
{8, 6} {3, 4}
{2, 5} {8, 7}
{5, 6} {3, 4}
{0, 4} {2, 5}
{5, 6} {3, 4}
{2, 4} {3, 5}
{1, 3} {2, 5}
{1, 4} {3, 6}
{5} {1, 3}
{5, 6} {0, 4}
{7} {5, 6}
{0, 5} {2, 4}
{7} {2, 3}
{8, 4} {0, 1}
{5} {1, 4}
{0, 1} {2, 5}
{0, 3} {1, 6}
{0, 1} {6, 7}
{2, 5} {1, 3}
{2, 3} {4, 5}
{4, 5} {0, 3}
{0, 6} {2, 3}
{1, 3} {0, 5}
{2, 5} {3, 4}
{1, 3} {4, 6}
{0, 1} {2, 5}
{1, 7} {0, 3}
{2, 7} {3, 6}
{2, 4} {6, 7}
{5} {1, 3}
{6} {3, 7}
{7} {1, 5}
{2, 5} {1, 3}
{0, 6} {3, 4}
{0, 6} {1, 2}
{7} {1, 4}
{7} {3, 4}
{8, 6} {5, 7}
{2, 5} {1, 4}
{4, 6} {3, 5}
{5, 6} {1, 4}
{4, 6} {2, 3}
{0, 3} {2, 6}
{0, 2} {3, 5}
{1, 4} {6, 7}
{8, 7} {1, 2}
{4} {0, 2}
{2, 3} {4}
{9, 2} {3, 4}
{0, 2} {4, 5}
{5} {3, 4}
{1, 3} {2, 5}
{1, 3} {8, 4}
{2, 4} {7}
{0, 1} {2, 3}
{1, 2} {4, 6}
{7} {2, 3}
{4, 5} {3, 6}
{5} {2, 3}
{3, 5} {2, 4}
{0, 3} {1, 2}
{0, 3} {4, 6}
{8, 6} 

Looks like the collision we have here the the diagonal pairs

In [12]:
# jc = jet collision
# jpc = jet pair collision
N_event = 0
N_jc_event = 0
N_jpc_event = 0

for h2b1b2_e, h3b1b2_e in zip(h2b1b2, h3b1b2):
    N_event += 1
    
    h2b1b2set = set(h2b1b2_e)
    h3b1b2set = set(h3b1b2_e)
    commonJetIndex = h2b1b2set.intersection(h3b1b2set)
    N_commonIndex = len(commonJetIndex)
    
    if N_commonIndex == 2:
        N_jpc_event +=1
    elif N_commonIndex == 1:
        N_jc_event += 1

print("jet pair collision event", N_jpc_event, "out of", N_event)
print("jet collision event", N_jc_event, "out of", N_event)

jet pair collision event 0 out of 42829
jet collision event 0 out of 42829


In [13]:
# jc = jet collision
# jpc = jet pair collision
N_event = 0
N_jc_event = 0
N_jpc_event = 0

for h1b1b2_e, h3b1b2_e in zip(h1b1b2, h3b1b2):
    N_event += 1
    
    h1b1b2set = set(h1b1b2_e)
    h3b1b2set = set(h3b1b2_e)
    commonJetIndex = h1b1b2set.intersection(h3b1b2set)
    N_commonIndex = len(commonJetIndex)
    
    if N_commonIndex == 2:
        N_jpc_event +=1
    elif N_commonIndex == 1:
        N_jc_event += 1

print("jet pair collision event", N_jpc_event, "out of", N_event)
print("jet collision event", N_jc_event, "out of", N_event)

jet pair collision event 0 out of 42829
jet collision event 0 out of 42829
